In [6]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv(r"classification.csv")
df.info()
import numpy as np

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Age              400 non-null    int64
 1   EstimatedSalary  400 non-null    int64
 2   Purchased        400 non-null    int64
dtypes: int64(3)
memory usage: 9.5 KB


In [2]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train=sc.fit_transform(x_train)
x_test= sc.transform(x_test)

# Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()


In [10]:
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.model_selection import GridSearchCV

In [12]:
parameter = {'bootstrap':[True,False],"n_estimators":[10,20,30,40,50,60,100,125]}
grid=GridSearchCV(classifier,parameter,cv=10)

In [13]:
grid.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 100, 125]})

In [14]:
rf_best = grid.best_estimator_

In [15]:
rf_best.score(x_test,y_test)

0.8916666666666667

# Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

# Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [18]:
params_grid = {'var_smoothing':np.logspace(0,-9,num=100)}
GNB=GaussianNB()
grid_nb = GridSearchCV(estimator=GNB,param_grid=params_grid,cv=10)
grid_nb.fit(x_train,y_train)


NameError: name 'GaussianNB' is not defined

# KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
parameters = {"n_neighbors": np.arange(1, 25)}


In [20]:
knn_g = GridSearchCV(knn, parameters, cv=5)

In [21]:
knn_g.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])})

# Support Vector Machine

In [14]:
from sklearn.svm import SVC

In [52]:
parameter = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','polynomial','gausian','linear']}
svc=SVC()
grid_svm = GridSearchCV(SVC(), parameter)
grid_svm.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'polynomial', 'gausian', 'linear']})

# Using Different Base Learners in single iteration


In [30]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier( n_estimators=50, random_state=0)
base_classifiers = {SVC():{'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','polynomial','gausian','linear']},
               KNeighborsClassifier():{"n_neighbors": np.arange(1, 25)},
               LogisticRegression(): {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]},
               RandomForestClassifier(): {'bootstrap':[True,False],"n_estimators":[10,20,30,40,50,60,100,125]},
               GaussianNB(): {'var_smoothing':np.logspace(0,-9,num=100)}}
for i in range(len(base_classifiers)):
    best_estimators=[]
    for k,j in base_classifiers.items():    
        Grid=GridSearchCV(k,j)
        md=Grid.fit(x_train, y_train)
        best_estimators.append(md.best_estimator_)
best_estimators

        
        



[SVC(C=1, gamma=1),
 KNeighborsClassifier(n_neighbors=3),
 LogisticRegression(),
 RandomForestClassifier(),
 GaussianNB(var_smoothing=0.04328761281083057)]

In [38]:
from sklearn.ensemble import BaggingClassifier
bag2 = BaggingClassifier( n_estimators=5, random_state=0)
base_classifiers = {SVC():{'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','polynomial','gausian','linear']},
               KNeighborsClassifier():{"n_neighbors": np.arange(1, 25)},
               LogisticRegression(): {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]},
               RandomForestClassifier(): {'bootstrap':[True,False],"n_estimators":[10,20,30,40,50,60,100,125]},
               GaussianNB(): {'var_smoothing':np.logspace(0,-9,num=100)}}
for i in range(len(base_classifiers)):
    bag2.estimators_=[]
    for k,j in base_classifiers.items():    
        Grid=GridSearchCV(k,j)
        md=Grid.fit(x_train, y_train)
        bag2.estimators_.append(md.best_estimator_)
print(bag2.estimators_)
        
        

[SVC(C=1, gamma=1), KNeighborsClassifier(n_neighbors=3), LogisticRegression(), RandomForestClassifier(n_estimators=60), GaussianNB(var_smoothing=0.04328761281083057)]


In [39]:
bag2.fit(x_train,y_train)

BaggingClassifier(n_estimators=5, random_state=0)

In [40]:
bag2.estimators_

[DecisionTreeClassifier(random_state=2087557356),
 DecisionTreeClassifier(random_state=132990059),
 DecisionTreeClassifier(random_state=1109697837),
 DecisionTreeClassifier(random_state=123230084),
 DecisionTreeClassifier(random_state=633163265)]

# Voting Classifier

In [24]:
print("knn: {}".format(knn_g.score(x_test, y_test)))
print("rf: {}".format(grid.score(x_test, y_test)))
print("log_reg: {}".format(lr.score(x_test, y_test)))
print("SVM: {}".format(grid_svm.score(x_test, y_test)))
print("Naive_Baes: {}".format(grid_nb.score(x_test, y_test)))

knn: 0.8916666666666667
rf: 0.8916666666666667
log_reg: 0.8166666666666667
SVM: 0.8916666666666667
Naive_Baes: 0.85


In [25]:
from sklearn.ensemble import VotingClassifier
estimators=[("knn", knn_g), ("rf", grid), ("log_reg", lr), ("SVM", grid_svm),("Naive_baes", grid_nb)]
ensemble = VotingClassifier(estimators, voting= "hard")

In [26]:
ensemble.fit(x_train,y_train)
ensemble.score(x_test,y_test)

0.8916666666666667